In [1]:
import pandas as pd
import sqlite3 as sql
from contextlib import closing

In [2]:
df = pd.read_csv("../data/Goodreads_books_with_genres.csv")
df = df.drop(axis=0, index=8180) # parche
df = df.drop(axis=0, index=11098) # parche
df["Author"] = df["Author"].str.replace('"',"'")
df['publication_date'] = pd.to_datetime(df['publication_date'], format='%m/%d/%Y')
df.head()

,Book Id,Title,Author,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,genres
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,2006-09-16,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,2004-09-01,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,2003-11-01,Scholastic,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,2004-05-01,Scholastic Inc.,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,2004-09-13,Scholastic,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Adve..."


Llenar tabla de géneros. Para eso necesito obtener todos los géneros individuales

In [4]:
series_genres = df['genres'].dropna()
list_genres = series_genres.str.split(pat=";").values.tolist()
# Flatten genres
flat_list_genres = [x2 for x1 in list_genres for x2 in x1]
# Capitalizar todos
flat_list_genres_cap = [x.capitalize().strip() for x in flat_list_genres]
array_genres_unique = pd.Series(flat_list_genres_cap).unique()
array_genres_unique


array(['Fantasy', 'Young adult', 'Fiction', 'Fantasy,magic', 'Childrens',
       'Adventure', 'Audiobook', 'Childrens,middle grade', 'Classics',
       'Science fiction fantasy', 'Fantasy,supernatural', 'Mystery',
       'Fantasy,paranormal', 'Novels', 'Paranormal,wizards',
       'Science fiction', 'Humor', 'Humor,comedy',
       'European literature,british literature', 'Nonfiction', 'Science',
       'History', 'Science,physics', 'Science,popular science',
       'Historical', 'Philosophy', 'Unfinished', 'Travel',
       'Cultural,africa', 'Autobiography,memoir', 'Eastern africa,kenya',
       'Biography', 'Travel,travelogue', 'Language,writing',
       'Humanities,language', 'Reference', 'Humanities,linguistics',
       'Language,words', 'Reference,dictionaries',
       'Writing,books about books', 'Cultural,australia',
       'Writing,essays', 'Literature,american',
       'The united states of america', 'Fantasy,epic fantasy',
       'Fantasy,high fantasy', 'Literature', 'Art', '

In [5]:
# Objeto de tipo género
class Genre:
    def __init__(self, genreId = 0, genre_name = None):
        self.genreId = genreId
        self.genre_name = genre_name


In [6]:
# Conexión a base de datos
conn = None
def connect():
    global conn
    if not conn: # if conn has not been set then set
        conn = sql.connect("../db/goodreads-db.sqlite")
        conn.row_factory = sql.Row # returns a dict instead of a tuple

In [7]:
# Agregar género
def add_genre(genre: Genre): # needs an object that represents all of the information of the author
    sql_query = '''INSERT OR IGNORE INTO Genres (GenreName)
    VALUES (?)'''
    with closing(conn.cursor()) as cursor:
        cursor.execute(sql_query, (genre.genre_name,))
        conn.commit()

In [8]:
connect()
for genre_name in array_genres_unique:
    genre = Genre(genre_name=genre_name)
    add_genre(genre)
print("Tabla de géneros llenada")

Tabla de géneros llenada


Ahora llenar la tabla BookAuthors

Para esto necesito primero leer los autores en base de datos, esto para obtener
sus ids.

Luego iterar por cada libro del del csv y, por cada autor de cada libro,
generar una nueva row en la tabla BookAuthors con el id del libro y el
id del autor

In [9]:
def get_authors_dictionary():
    sql_query = '''SELECT * FROM Authors'''
    df_authors = pd.read_sql_query(sql_query, conn)
    authors_dict = {}
    for index, row in df_authors.iterrows():
        authors_dict[row["AuthorName"]] = int(row["AuthorID"])
    return authors_dict

In [10]:
# Objeto de tipo BookAuthor
class BookAuthor:
    def __init__(self, book_id = None, author_id = None):
        self.bookId = book_id
        self.authorId = author_id

In [11]:
# Agregar BookAuthor
def add_bookAuthor(bookAuthor: BookAuthor):
    sql_query = '''INSERT OR IGNORE INTO BookAuthors (BookID, AuthorID)
    VALUES (?,?)'''
    with closing(conn.cursor()) as cursor:
        cursor.execute(sql_query, (int(bookAuthor.bookId), int(bookAuthor.authorId)))
        conn.commit()

In [12]:
authors_dict = get_authors_dictionary()
for index, row in df.iterrows():
    book_id = row["Book Id"]
    authors = row['Author']
    authors_array = authors.split("/")
    # Por cada autor, obtener su id
    for author in authors_array:
        # Quitar espacios extra
        author = " ".join(author.split())
        author_id = authors_dict.get(author)
        if(author_id != None):
            # Crear registro en la tabla BookAuthors
            bookAuthor = BookAuthor(book_id, author_id)
            add_bookAuthor(bookAuthor)
        else:
            print(f"Author {author} not found!")
print("Tabla de BookAuthors llenada")

Tabla de BookAuthors llenada


Llenar tabla BookGenres



In [13]:
def get_genres_dictionary():
    sql_query = '''SELECT * FROM Genres'''
    df_genres = pd.read_sql_query(sql_query, conn)
    genres_dict = {}
    for index, row in df_genres.iterrows():
        genres_dict[row["GenreName"]] = int(row["GenreID"])
    return genres_dict

In [14]:
# Objeto de tipo BookGenre
class BookGenre:
    def __init__(self, book_id = None, genre_id = None):
        self.bookId = book_id
        self.genreId = genre_id

In [15]:
# Agregar BookGenre
def add_bookGenre(bookGenre: BookGenre):
    sql_query = '''INSERT OR IGNORE INTO BookGenres (BookID, GenreID)
    VALUES (?,?)'''
    with closing(conn.cursor()) as cursor:
        cursor.execute(sql_query, (bookGenre.bookId, bookGenre.genreId))
        conn.commit()

In [16]:
genres_dictionary = get_genres_dictionary()
for index, row in df.iterrows():
    book_id = row["Book Id"]
    genres = row['genres']
    if(pd.isna(genres)):
        continue
    genres_array = genres.split(";")
    genres_array = [x.capitalize().strip() for x in genres_array]
    # Por cada autor, obtener su id
    for genre in genres_array:
        genre_id = genres_dictionary.get(genre)
        if(genre_id != None):
            # Crear registro en la tabla BookAuthors
            bookGenre = BookGenre(book_id, genre_id)
            add_bookGenre(bookGenre)
        else:
            print(f"Genre {genre} not found!")        

print("Tabla de BookGenres llenada")

Tabla de BookGenres llenada
